# Week 3

### Context
#### k-Fold Technic
- Label Postprocess only binary classification

#### Feature Selection
- Permutation Importance

#### AutoML
+ NNI

In [1]:
import os
from os.path import join

import multiprocessing
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

n_cpus = multiprocessing.cpu_count()

In [2]:
BASE_DIR = './'

train_path = join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

label = data['credit']

In [3]:
data.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [4]:
data.shape

(26457, 20)

In [5]:
data.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000000,26457.000000,2.645700e+04,26457.000000,26457.000000,26457.0,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000
mean,13228.000000,0.428658,1.873065e+05,-15958.053899,59068.750728,1.0,0.224742,0.294251,0.091280,2.196848,-26.123294,1.519560
std,7637.622372,0.747326,1.018784e+05,4201.589022,137475.427503,0.0,0.417420,0.455714,0.288013,0.916717,16.559550,0.702283
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6614.000000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19842.000000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,19.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,2.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [7]:
test.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
0,26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0
1,26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0
2,26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0
3,26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0
4,26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0


In [8]:
test.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month
count,10000.00000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,31456.50000,0.434700,1.850433e+05,-16020.466400,59776.690400,1.0,0.227600,0.296300,0.085600,2.202700,-26.272400
std,2886.89568,0.729102,1.015398e+05,4197.672887,138121.224504,0.0,0.419304,0.456648,0.279786,0.898272,16.348557
min,26457.00000,0.000000,2.700000e+04,-25152.000000,-15661.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000
25%,28956.75000,0.000000,1.215000e+05,-19483.250000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000
50%,31456.50000,0.000000,1.575000e+05,-15606.000000,-1577.000000,1.0,0.000000,0.000000,0.000000,2.000000,-25.000000
75%,33956.25000,1.000000,2.250000e+05,-12539.000000,-410.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000
max,36456.00000,5.000000,1.575000e+06,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,7.000000,0.000000


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          10000 non-null  int64  
 1   gender         10000 non-null  object 
 2   car            10000 non-null  object 
 3   reality        10000 non-null  object 
 4   child_num      10000 non-null  int64  
 5   income_total   10000 non-null  float64
 6   income_type    10000 non-null  object 
 7   edu_type       10000 non-null  object 
 8   family_type    10000 non-null  object 
 9   house_type     10000 non-null  object 
 10  DAYS_BIRTH     10000 non-null  int64  
 11  DAYS_EMPLOYED  10000 non-null  int64  
 12  FLAG_MOBIL     10000 non-null  int64  
 13  work_phone     10000 non-null  int64  
 14  phone          10000 non-null  int64  
 15  email          10000 non-null  int64  
 16  occyp_type     6848 non-null   object 
 17  family_size    10000 non-null  float64
 18  begin_m

In [10]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'],           inplace=True)

In [11]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']

Numeric Columns: 
['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'family_size', 'begin_month']


#### 라벨 데이터 인코딩

In [12]:
label = label.astype(int)

#### 전처리 프로세스 함수로 작성

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    imputer = SimpleImputer(strategy='most_frequent')
    tmp_x_train[cat_columns] = imputer.fit_transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_x_train[cat_columns])
    
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

## k-Fold technic
### Label Postprocessing
- 이진 분류인 경우 Threshold 값을 최적화하여 조금 더 좋은 성능을 이끌어낼 수 있습니다. 
- k-Fold를 활용하여 train 라벨에서 가장 좋은 threshold 값을 찾아, 각 라벨 생성을 최적화 할 수 있습니다.

1. train 셋에 대해서도 라벨을 모읍니다.
2. y_train, y_train_pred 값으로 최적 threshold 값을 찾습니다.
3. threshold 를 0.01 단위로 0~1 사이값을 변경해가면서 평가 지표에 대해 평가합니다.
4. 그 중 가장 높은 평가 지표를 갖는 threshold를 선택합니다.

### 클래스가 0, 1인 데이터만 추출하겠습니다.

In [14]:
binary_data = data.loc[label < 2,:]
binary_label = label.loc[label < 2]

In [15]:
binary_data.reset_index(inplace=True, drop=True)
binary_label.reset_index(inplace=True, drop=True)

In [16]:
binary_data.shape, binary_label.shape

((9489, 18), (9489,))

In [17]:
from sklearn.model_selection import StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import log_loss, accuracy_score

val_scores = list()
oof_train = np.zeros((binary_data.shape[0], 2))
oof_pred  = np.zeros((test.shape[0], 2))

for i, (trn_idx, val_idx) in enumerate(skf.split(binary_data, binary_label)):
    x_train, y_train = binary_data.iloc[trn_idx, :], binary_label.iloc[trn_idx,]
    x_valid, y_valid = binary_data.iloc[val_idx, :], binary_label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=n_cpus-1)
    
    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 accuracy_score 확인
    trn_acc = accuracy_score(y_train, (model.predict_proba(x_train)[:, 1] > 0.5).astype(int))
    val_acc = accuracy_score(y_valid, (model.predict_proba(x_valid)[:, 1] > 0.5).astype(int))
    print('{} Fold, train accuracy_score : {:.4f}4, validation accuracy_score : {:.4f}'.format(i, trn_acc, val_acc))
    
    val_scores.append(val_acc)
    
    _, x_data, _ = preprocess(binary_data.iloc[trn_idx, :], binary_data, test)
    oof_train += model.predict_proba(x_data) / 5
    oof_pred  += model.predict_proba(x_test) / 5
    

# 교차 검증 accuracy_score 평균 계산하기
print('Cross Validation Score : {:.5f}'.format(np.mean(val_scores)))

0 Fold, train accuracy_score : 0.70644, validation accuracy_score : 0.6781
1 Fold, train accuracy_score : 0.70324, validation accuracy_score : 0.6770
2 Fold, train accuracy_score : 0.70194, validation accuracy_score : 0.6786
3 Fold, train accuracy_score : 0.70954, validation accuracy_score : 0.6733
4 Fold, train accuracy_score : 0.70114, validation accuracy_score : 0.6784
Cross Validation Score : 0.67710


In [19]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import f1_score

def threshold_search(y_true, y_proba):
    # 0 ~ 1 사이에서 0.01 단위로 threhold 생성
    thresholds = np.linspace(0, 1, 101) 

    # 각 threshold 마다 f1_score 측정합니다.
    f1_scores   = np.array([ f1_score(y_true, (y_proba > t).astype(np.int)) 
                             for t in thresholds ])
    best_score = np.max(f1_scores)

    # 가장 높은 f1_score를 갖는 threshold를 택합니다.
    best_th = thresholds[np.argmax(f1_scores)]
    return best_th, best_score

def scoring(y_true, y_proba, verbose=True):
    # 반복적으로 kFold를 진행할 수 있습니다.
    rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

    scores = []
    ths = []
    for train_index, test_index in rkf.split(y_true, y_true):
        y_prob_train, y_prob_test = y_proba[train_index], y_proba[test_index]
        y_true_train, y_true_test = y_true[train_index], y_true[test_index]

        best_threshold, sc = threshold_search(y_true_train, y_prob_train)
        
        ths.append(best_threshold)
        scores.append(sc)

    # 최적 threshold들의 평균 값으로 최종 threshold를 택합니다.
    best_th = np.mean(ths)
    score = np.mean(scores)

    if verbose: print(f'Best threshold: {np.round(best_th, 4)}, Score: {np.round(score, 5)}')

    return best_th, score

In [20]:
best_th, _ = scoring(binary_label, oof_train[:, 1])

Best threshold: 0.6192, Score: 0.87144


#### 찾은 threhold 값으로 Validation Score 재 측정

In [21]:
val_scores = list()

for i, (trn_idx, val_idx) in enumerate(skf.split(binary_data, binary_label)):
    x_train, y_train = binary_data.iloc[trn_idx, :], binary_label.iloc[trn_idx,]
    x_valid, y_valid = binary_data.iloc[val_idx, :], binary_label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=n_cpus-1)
    
    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 accuracy_score 확인
    trn_acc = accuracy_score(y_train, (model.predict_proba(x_train)[:, 1] > best_th).astype(int))
    val_acc = accuracy_score(y_valid, (model.predict_proba(x_valid)[:, 1] > best_th).astype(int))
    print('{} Fold, train accuracy_score : {:.4f}4, validation accuracy_score : {:.4f}'.format(i, trn_acc, val_acc))
    
    val_scores.append(val_acc)

# 교차 검증 accuracy_score 평균 계산하기
print('Cross Validation Score : {:.5f}'.format(np.mean(val_scores)))

0 Fold, train accuracy_score : 0.83644, validation accuracy_score : 0.7002
1 Fold, train accuracy_score : 0.83114, validation accuracy_score : 0.7060
2 Fold, train accuracy_score : 0.83844, validation accuracy_score : 0.7034
3 Fold, train accuracy_score : 0.83274, validation accuracy_score : 0.6976
4 Fold, train accuracy_score : 0.83024, validation accuracy_score : 0.7116
Cross Validation Score : 0.70376


## Feature Selection
### Permutation Importance
- 좋은 변수를 선택하는 방법 중 최근에 많이 사용되는 Permutation Importance에 대해 알아보겠습니다. 
- Permutation Importance의 기본 원리는 어떤 변수를 임의로 섞어 그 중 타겟 변수에 영향을 많이 주는 변수를 탐색합니다.
- 예를 들어 타겟 변수를 예측하기 좋은 변수는 임의로 섞었을 경우 모델의 성능이 많이 떨어지게되는데, 이러한 방식으로 좋은 변수를 선별해 냅니다.

$$ i_j=s-{{1}\over{K}}\sum^K_{k=1}s_{k,j}$$

$$ s=원본\ 데이터셋의\ 점수 [분류(Accuracy)| 회귀(R^2)] $$
$$ j=변수(feature)\ 인덱스$$
$$ k=반복\ 인덱스$$



#### ref
Permutation Importance: https://scikit-learn.org/stable/modules/permutation_importance.html

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
x_train, x_valid, y_train, y_valid = train_test_split(data, label, 
                                                      test_size=0.3,
                                                      random_state=42,
                                                      shuffle=True)

In [24]:
x_train, x_valid, _ = preprocess(x_train, x_valid, test)

In [25]:
from sklearn.inspection import permutation_importance

model = RandomForestClassifier(random_state=42, n_jobs=(n_cpus-1))
model.fit(x_train, y_train)

r = permutation_importance(model, x_valid, y_valid,
                           n_repeats=10,
                           random_state=42)

In [26]:
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{x_valid.columns[i]:<8}: "
               f"{r.importances_mean[i]:.3f}"
               f" +/- {r.importances_std[i]:.3f}")

begin_month: 0.069 +/- 0.003
DAYS_BIRTH: 0.005 +/- 0.002
20      : 0.001 +/- 0.000
27      : 0.001 +/- 0.000
39      : 0.001 +/- 0.000
24      : 0.000 +/- 0.000
44      : 0.000 +/- 0.000
42      : 0.000 +/- 0.000
11      : 0.000 +/- 0.000
40      : 0.000 +/- 0.000
